#### 데이터 로드하기

In [2]:
import pandas as pd
import numpy as np
import os

def load_hotel_reserve():
  customer_tb = pd.read_csv('./data/customer.csv')
  hotel_tb = pd.read_csv('./data/hotel.csv')
  reserve_tb = pd.read_csv('./data/reserve.csv')
  return customer_tb, hotel_tb, reserve_tb


def load_holiday_mst():
  holiday_tb = pd.read_csv('./data/holiday_mst.csv',
                           index_col=False)
  return holiday_tb


def load_production():
  production_tb = pd.read_csv('./data/production.csv')
  return production_tb


def load_production_missing_num():
  production_tb = pd.read_csv('./data/production_missing_num.csv')
  return production_tb


def load_production_missing_category():
  production_tb = pd.read_csv('./awesomebook-master/data/production_missing_category.csv')
  return production_tb


def load_monthly_index():
  monthly_index_tb = pd.read_csv('./data/monthly_index.csv')
  return monthly_index_tb


def load_meros_txt():
  with open('./data/txt/meros.txt', 'r') as f:
    meros = f.read()
    f.close()
  return meros


In [3]:
production_tb = load_production()

### 4. 학습용과 검증용 데이터 나누기

목적
* 예측 모델 평가하기 위해 사용한다

어떻게 분할?
* 학습 데이터, 검증 데이터로 나눈다.
* 학습/검증 데이터는 같은 방식으로 분할한다 (아니면 도메인이 달라져서 이상한 결과가 나올 수도 있다)
* 예측 모델에 넣기 직전에 분할한다

적용 데이터는 답을 알 수 없는 상태에서 사용하는 데이터로 분할할 필요 없다.

교차 검증   
데이터를 여러 세트로 나눈 후, 한 개는 마지막 검증용, 나머지 데이터셋은 학습용으로 이 학습용 데이터를 다시 학습/검증 데이터로 나눠 학습한다.

In [ ]:
# %pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.1 MB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 30.3 MB 17.3 MB/s eta 0:00:01
     |████████████████████████████████| 309 kB 46.0 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


#### 1) 교차 검증 데이터 분할

In [8]:
# 1. 제조 데이터 로드
production_tb = load_production()
production_tb.head(2)

# fault_flg = falut flag로 해당 데이터의 레이블, 정답

,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False


train_data, test_data, train_target, test_target = train_test_split( data, target , ...)   
* data : 학습에 사용할 입력 데이터
* target : 모델이 맞춰야 할 정답

* train_data, test_data : 각각 학습에서 사용할 훈련/검증 데이터
* train_target, test_target : 각각 검증에서 사용할 훈련/검증 데이터


In [ ]:
# 2. 데이터 분할 패키지 로드 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# 3. 데이터 분할 

train_data, test_data, train_target, test_target = \
    train_test_split(production_tb.drop('fault_flg', axis=1),   # 모델학습을 위해 정답을 제거
                     production_tb[['fault_flg']],              # 정답만 추출
                     test_size=0.2)                             # 학습 80%, 테스트 20%로 분할

train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)
train_target.reset_index(inplace=True, drop=True)
test_target.reset_index(inplace=True, drop=True)

row_no_list = list(range(len(train_target)))

k_fold = KFold(n_splits=4, shuffle=True)

for train_cv_no, test_cv_no in k_fold.split(row_no_list):
    train_cv = train_data.iloc[train_cv_no, :]
    test_cv = train_data.iloc[test_cv_no, :]

#### 2) 교차 검증 - 시간 데이터 분할

교차 검증시 시간 데이터를 이용할 경우 과거데이터로 학습하고 미래 데이터로 검증하여야 한다.

In [10]:
monthly_index_tb = load_monthly_index()

train_window_start = 1
train_window_end = 24
horizon = 12
skip = 12

monthly_index_tb.sort_values(by='year_month')

while True:
    test_window_end = train_window_end + horizon
    train = monthly_index_tb[train_window_start:train_window_end]
    test = monthly_index_tb[(train_window_end + 1):test_window_end]
    if test_window_end >= len(monthly_index_tb.index):
        break
    train_window_start += skip
    train_window_end += skip